In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import preprocess_vehicles_data

# Filtering to get data for La Baule

#### VP (Voitures Particulières)

In [51]:
vp = pd.read_excel(f"../data/EV/parc_vp_com_2011_2024.xlsx", header=3,)
vp = vp.rename(columns={'2 024': 2024})
# vp = vp.rename(columns={'2 024': '2024'})
# vp.shape
vp.head()

,Code commune de résidence,libellé commune de résidence,Carburant,statut,Crit'Air,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,00000,Inconnu,Essence hybride rechargeable,PAR,Crit'Air 1,0,25,36,30,20,12,6,5,4,4,12,59,53.0,18.708
1,00000,Inconnu,Diesel thermique,PRO,Crit'Air 4,59,57,57,54,53,52,38,28,24,19,6,4,4.0,2.040
2,00000,Inconnu,Gaz,PRO,Crit'Air 1,0,0,0,0,1,1,1,1,1,1,0,0,0.0,0.000
3,00000,Inconnu,Essence hybride rechargeable,PRO,Crit'Air 1,0,5,10,2,1,0,0,0,0,0,5,10,15.0,13.000
4,00000,Inconnu,Essence thermique,PRO,Crit'Air 3,78,74,70,74,76,72,50,37,35,24,2,0,0.0,0.000


In [52]:
vp_baule = vp[vp['Code commune de résidence'] == '44055']
# vp_baule.to_pickle("../data/EV/vp_baule_2011_2024.pkl")
vp_baule.to_excel("../data/EV/preprocessed/vp_baule_2011_2024.xlsx", index=False)

#### VUL (Véhicules Utilitaires Légers)

In [27]:
vul = pd.read_excel(f"../data/EV/parc_vul_com_2011_2024.xlsx", header=3,)
vul_baule = vul[vul['Code commune de résidence'] == '44055']
# vul_baule.head()
vul_baule.to_excel("../data/EV/preprocessed/vul_baule_2011_2024.xlsx", index=False)

### PL (Poids Lourds)

In [28]:
pl = pd.read_excel(f"../data/EV/parc_pl_com2011_2024.xlsx", header=3,)
pl_baule = pl[pl['Code commune de résidence'] == '44055']
# pl_baule.head()
pl_baule.to_excel("../data/EV/preprocessed/pl_baule_2011_2024.xlsx", index=False)

### TCP (Transport en Commun Collectif de Personnes)

In [29]:
tcp = pd.read_excel(f"../data/EV/parc_tcp_commune_2024.xlsx", header=3,)
tcp_baule = tcp[tcp['Code commune de résidence'] == '44055']
tcp_baule.head()
tcp_baule.to_excel("../data/EV/preprocessed/tcp_baule_2024.xlsx", index=False)

# Processing to get EVs

In [53]:
vul_baule = pd.read_excel(f"../data/EV/preprocessed/vul_baule_2011_2024.xlsx")
vp_baule = pd.read_excel(f"../data/EV/preprocessed/vp_baule_2011_2024.xlsx")
tcp_baule = pd.read_excel(f"../data/EV/preprocessed/tcp_baule_2024.xlsx")
pl_baule = pd.read_excel(f"../data/EV/preprocessed/pl_baule_2011_2024.xlsx")

In [54]:
raw_vehicules = {
    'vp': vp_baule,
    'vul': vul_baule, 
    'pl': pl_baule,
    'tcp': tcp_baule
}

In [55]:
process_vehicles = preprocess_vehicles_data(raw_vehicules)

100%|██████████| 4/4 [00:00<00:00, 77.21it/s]


In [56]:
process_vehicles.head()

Carburant,Code_Commune,Nom_Commune,Year,Statut,Type_Vhcl,NB_THERMIQUE,NB_RECHARGEABLE,NB_EL
0,44055,La Baule-Escoublac,2024,Particulier,vp,12455.083,207.407,301.049
1,44055,La Baule-Escoublac,2024,Professionnel,vp,547.423,93.178,58.000
2,44055,La Baule-Escoublac,2011,Particulier,vp,11481.000,14.000,1.000
3,44055,La Baule-Escoublac,2011,Professionnel,vp,489.000,3.000,3.000
4,44055,La Baule-Escoublac,2012,Particulier,vp,11538.000,25.000,1.000


### Merging all types of vehicules for each year (thermique / rechargeable / Electrique)

In [57]:
# df_grouped = process_vehicles.groupby("Year", as_index=False)[
#     ["NB_VP_THERMIQUE", "NB_VP_RECHARGEABLE", "NB_VP_EL"]
# ].sum()

df_grouped = process_vehicles.groupby("Year", as_index=False)[
    ["NB_THERMIQUE", "NB_RECHARGEABLE", "NB_EL"]
].sum()

df_grouped

Carburant,Year,NB_THERMIQUE,NB_RECHARGEABLE,NB_EL
0,2011,13440.000,17.000,10.000
1,2012,13441.000,27.000,10.000
2,2013,13526.000,30.000,16.000
3,2014,13565.000,29.000,16.000
4,2015,13614.000,23.000,21.000
5,2016,13670.000,26.000,36.000
6,2017,13884.000,26.000,47.000
7,2018,14175.000,32.000,56.000
8,2019,14486.000,40.000,67.000
9,2020,14789.000,49.000,93.000


In [64]:
df_2024 = df_grouped[df_grouped['Year'] == '2024']
nb_voitures_baule_2024 = (df_2024['NB_EL'] + df_2024['NB_RECHARGEABLE'] + df_2024['NB_THERMIQUE']).iloc[0]
ratio_baule_2024 = ((df_2024['NB_EL'] + df_2024['NB_RECHARGEABLE']) / (df_2024['NB_EL'] + df_2024['NB_RECHARGEABLE'] + df_2024['NB_THERMIQUE'])).iloc[0]
print(ratio_baule_2024)
print(nb_voitures_baule_2024)

0.04590522952745761
15267.497999999998


# Getting Ratio for a region

Note
* En France, le taux global est de 2-3%
* Avec ce dataset on peut avoir le taux dans la région de la Baule pour mieux estimer la demande du traffic liée aux EVs

In [61]:
vp_24 = vp[['Code commune de résidence', 'libellé commune de résidence', 'Carburant', 'statut', 2024]]
tcp_24 = tcp[['Code commune de résidence', 'libellé commune de résidence', 'Carburant', 'statut', 2024]]
pl_24 = pl[['Code commune de résidence', 'libellé commune de résidence', 'Carburant', 'statut', 2024]]
vul_24 = vul[['Code commune de résidence', 'libellé commune de résidence', 'Carburant', 'statut', 2024]]

# TODO:
* Faire une fonction qui calcule automatiquement à partir du code de la ville
* Faire une démo